# SCRIPT 02: Create Segmentation and Generate LULC Statistics

This is the second script used in the methodology. Here, the a segmentation is created with optical reduction for the year. Then, statistics are created for every polygon using LULC data from Google Dynamic world, ESA World Cover, and MapBiomas. These statistics are used to generate data regarding the agreement between these maps for the class Agriculture. Combining three LULC projects obviously do not yeld a perfect map, therefore, a posterior step must be conducted to filter the samples generated with this map as reference.

In the following cells, please refer to the comments in the code for further explanations of its functioning.

In [ ]:
# importing packages
import time
from rsgislib.segmentation import shepherdseg
import rsgislib.segmentation as seg
import rasterio as r
import rasterio
from rasterio.features import shapes
import numpy as np
import geopandas as gp
import matplotlib
import numpy as np
from rasterio.mask import mask
import shapely
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import random
import subprocess

In [ ]:
# defining the function to write the log to a file
def write_text(text, print_end='\n'):
    with open("/home/bruno.matosak/Semiarido/segmentation_log.txt", 'a+') as f:
        print(text, end=print_end, flush=True)
        f.write(text+print_end)

In [ ]:
# in this cell, the function that makes the segmentation is defined
def do_the_segmentation(input_img, ref_path, save_directory, save_file_name):
    t1 = time.time()
    # DOING THE SEGMENTATION
    # write initial log messages
    write_text('------------------------------------------------------------------------------------------')
    write_text('MAKING THE SEGMENTATION WITH RSGISLIB...\n')
    
    # reimport important packages
    from rsgislib.segmentation import shepherdseg
    import rsgislib.segmentation as seg
    
    # defines path of output file to save segmentation
    out_clumps_img = os.path.join(save_directory, f'{save_file_name}.tif')
    
    # write log messages
    write_text(f'Input Image: {input_img}')
    write_text(f'Output Clumps Image: {out_clumps_img}')
    write_text(f'Save File Name: {save_file_name}')
    write_text(f'Reference Path: {ref_path}\n')

    # runs the segmentation for the given file
    shepherdseg.run_shepherd_segmentation(input_img,
                                          out_clumps_img,
                                          out_mean_img=None,
                                          tmp_dir=save_directory, 
                                          gdalformat='KEA',
                                          calc_stats=False,
                                          no_stretch=False,
                                          no_delete=False,
                                          num_clusters=60,
                                          min_n_pxls=100, 
                                          dist_thres=100, 
                                          bands=None, 
                                          sampling=100, 
                                          km_max_iter=200, 
                                          process_in_mem=False, 
                                          save_process_stats=False, 
                                          img_stretch_stats='', 
                                          kmeans_centres='', 
                                          img_stats_json_file='')
    
    # computes elapsed time and writes it in log file
    t2 = time.time()
    write_text("Elapsed time from start: %.3f minutes." % ((t2-t1)/60))

    # GEORREFERENCING CLUMPS OF IMAGES
    # write some messages to log file
    write_text('------------------------------------------------------------------------------------------')
    write_text('GEORREFERENCING THE CLUMPS IMAGE...\n')
    
    # reimport important library
    import rasterio as r
    
    # defines save file geodata profile
    profile_out = r.open(input_img).profile
    profile_in = r.open(out_clumps_img).profile
    img = r.open(out_clumps_img).read(1)
    
    # output raster path
    out_name_geo = out_clumps_img[:-4]+'_geo.tif'
    
    # update profile with desired defined parameters
    profile_out.update({'dtype': profile_in['dtype'], 'count': profile_in['count'], 'compress': 'packbits'})
    
    # write raster to file
    with r.open(out_name_geo, 'w', **profile_out) as dst:
        dst.write(img, 1)

    # computes elapsed time and writes it to log file
    t2 = time.time()
    write_text("Elapsed time from start: %.3f minutes." % ((t2-t1)/60))

    # POLYGONIZE RASTER
    # write some log messages
    write_text('------------------------------------------------------------------------------------------')
    write_text('POLYGONIZING RASTER...\n')
    
    # importing packages again
    import rasterio
    from rasterio.features import shapes
    import numpy as np
    import geopandas as gp
    
    # opens reference to copy profile
    ref = r.open(ref_path)
    
    # assign None to mask
    mask = None
    
    # open file and polygonize shapes
    with rasterio.Env():
        with rasterio.open(out_name_geo) as src:
            image = np.asarray(src.read(1), dtype = np.int32) # first band
            results = (
            {'properties': {'raster_val': v}, 'geometry': s}
            for i, (s, v) 
            in enumerate(
                shapes(image, mask=mask, transform=src.transform)))
    
    # gets shapes in list
    geoms = list(results)
    
    # creates geopandas GeoDataFrame with shapes and reproject it to nice CRS
    gpd_polygonized_raster = gp.GeoDataFrame.from_features(geoms)
    gpd_polygonized_raster = gpd_polygonized_raster.set_crs(r.open(input_img).crs)

    # reimporting some packages
    import matplotlib
    import numpy as np
    
    # defining colors and plotting result to see if everything is ok
    cmap = matplotlib.colors.ListedColormap(np.random.rand(256,3))
    gpd_polygonized_raster.plot(cmap=cmap)

    # compute elapsed time and write it to log file
    t2 = time.time()
    write_text("Elapsed time from start: %.3f minutes." % ((t2-t1)/60))

    # ITERATING THROUGH EVERY POLYGON AND CALCULATING ITS LULC STATISTICS
    # writing message to log file
    write_text('------------------------------------------------------------------------------------------')
    write_text('ITERATING THROUGH EVERY POLYGON...\n')
    
    # reimporting packages
    import rasterio as r
    from rasterio.mask import mask
    import shapely
    import numpy as np
    import matplotlib.pyplot as plt

    # defining loop parameters for percentage in log
    percent_iter = 5
    print_percent = 0
    count = 0
    total=len(gpd_polygonized_raster)
    
    # loop to analyse every polygon
    for index, row in gpd_polygonized_raster.iterrows():
        # gets data from LULC raster using polygon contours
        data = np.asarray(mask(ref, [shapely.geometry.mapping(row.geometry)], crop=True, nodata=255)[0], dtype = np.float32)
        
        # statistics for each polygon.
        # qt: quantity
        # 0: not agriculture
        # 1: agriculture
        # GDC: Google Dynamic World
        # ESA: ESA World Cover
        # MB: MapBiomas
        qt_0_GDC = np.sum(data[0]==0)
        qt_0_ESA = np.sum(data[1]==0)
        qt_0_MB  = np.sum(data[2]==0)
        qt_1_GDC = np.sum(data[0]==1)
        qt_1_ESA = np.sum(data[1]==1)
        qt_1_MB  = np.sum(data[2]==1)

        # save statistics to GeoDataFrame
        gpd_polygonized_raster.at[index, 'qt_0_GDC'] = qt_0_GDC
        gpd_polygonized_raster.at[index, 'qt_0_ESA'] = qt_0_ESA
        gpd_polygonized_raster.at[index, 'qt_0_MB']  = qt_0_MB
        gpd_polygonized_raster.at[index, 'qt_1_GDC'] = qt_1_GDC
        gpd_polygonized_raster.at[index, 'qt_1_ESA'] = qt_1_ESA
        gpd_polygonized_raster.at[index, 'qt_1_MB']  = qt_1_MB

        # statistics for each polygon, part 2
        # total amount of each class (agriculture, not agriculture) for the polygon
        # GEM: considering all three sources (Google, ESA, MapBiomas)
        # GM: considering only Google and Mapbiomas
        qt_0_GEM = qt_0_GDC + qt_0_ESA + qt_0_MB
        qt_1_GEM = qt_1_GDC + qt_1_ESA + qt_1_MB
        qt_0_GM = qt_0_GDC + qt_0_MB
        qt_1_GM = qt_1_GDC + qt_1_MB

        #save statistics to GeoDataFrame, part 2
        gpd_polygonized_raster.at[index, '0_GEM']  = qt_0_GEM
        gpd_polygonized_raster.at[index, '1_GEM']  = qt_1_GEM
        gpd_polygonized_raster.at[index, '0_GM']  = qt_0_GM
        gpd_polygonized_raster.at[index, '1_GM']  = qt_1_GM

        # computes which class has the majority in the polygon and also
        # a 'reliability' metric.
        # max: class with majority in polygon
        # conf: reliability of the class with majority in polygon. Represents the percentage of
        #       the majority class inside polygon.
        if qt_0_GEM < qt_1_GEM:
            max_GEM = 1
            conf_GEM = 100*qt_1_GEM/(qt_0_GEM+qt_1_GEM)
        else:
            max_GEM = 0
            conf_GEM = 100*qt_0_GEM/(qt_0_GEM+qt_1_GEM)

        if qt_0_GM < qt_1_GM:
            max_GM = 1
            conf_GM = 100*qt_1_GM/(qt_0_GM+qt_1_GM)
        else:
            max_GM = 0
            conf_GM = 100*qt_0_GM/(qt_0_GM+qt_1_GM)

        # writes the class number with majority to GeoDataFrame
        # also writes the reliability metric.
        gpd_polygonized_raster.at[index, 'max_GEM']  = max_GEM
        gpd_polygonized_raster.at[index, 'conf_GEM'] = conf_GEM
        gpd_polygonized_raster.at[index, 'max_GM']   = max_GM
        gpd_polygonized_raster.at[index, 'conf_GM']  = conf_GM

        # adds to polygon count
        count += 1

        # writes the percentage of completion of the process
        if 100*count/total >= print_percent:
            write_text(f'{print_percent}...', print_end='')
            print_percent+=percent_iter
    
    # writes log messages
    write_text('')
    write_text('Saving... ')
    gpd_polygonized_raster.to_file(os.path.join(save_directory, f'{save_file_name}.shp'))
    write_text('Done!')

    # computes elapsed time and writes it to log file
    t2 = time.time()
    write_text("Elapsed time from start: %.3f minutes." % ((t2-t1)/60))

    # everything is done. writing messages to log.
    write_text('------------------------------------------------------------------------------------------')
    write_text('ALL DONE (for now)\n')

    # computing elapsed time and write it to log
    t2 = time.time()
    write_text("Elapsed time from start: %.3f minutes." % ((t2-t1)/60))

In [ ]:
# in this cell, there is a loop that is used to apply the segmentation to all tiles, with
# the cell defined in the previous cell

# start time
t_start = time.time()
    
# load files to be segmented, creating a list with them
files = glob.glob('/home/bruno.matosak/Semiarido/MultiInput/LULC/LULC_id*.tif')
random.shuffle(files)

# eliminating files that have already been processed from the list
files_done = glob.glob('/home/bruno.matosak/Semiarido/MultiInput/segmentations/segmentation_id*.shp')
for file in files_done:
    file2 = file.replace('segmentations/segmentation', 'LULC/LULC').replace('.shp', '.tif')
    files.remove(file2)

# iterating through all the files to create the reference
count = 1
for file in files:
    # gets the tile's id from the file currently being processed
    id_ = file.split('d')[-1].split('.')[0]
    
    # writes messages to log file
    write_text('==========================================================================================')
    write_text(f'SEGMENTING ID:{id_} ({count}/{len(files)})')
    
    # the big boi, our segmentation
    do_the_segmentation(input_img = file.replace('/LULC/', '/yearly_reduction/').replace('/LULC_', '/Reduction_Optical_Year_'),
                        ref_path = file, 
                        save_directory = '/home/bruno.matosak/Semiarido/MultiInput/segmentations',
                        save_file_name = f'segmentation_id{id_}')
    
    # all done!
    write_text('\n\n')
    count += 1

# writes message to log file
write_text('ALL DONE.')

# computes total elapsed time and writes it to log file
t_end = time.time()
write_text("Total elapsed time: %.3f hours." % ((t_end-t_start)/3600))